In [ ]:
# Set up the workspace
using SumOfSquares, JuMP, PolyJuMP, DynamicPolynomials, MultivariatePolynomials
using Mosek#, CSDP#, SCS
using Plots
gr()

include("../src/NormalSoS.jl")
using NormalSoS

# Linear Examples

As a first test case we will examine the linear example of Zhou _et al._ (2012). For a linear system the dynamics is expressed in terms of a matrix $A$, and the properties of the decomposition are determined by some well known properties of this matrix.
\begin{equation}
    \dot{x} = f(x) = Ax.
\end{equation}
If $A$ is a normal matrix, then an orthogonal decomposition should be achieveable via a symmetric-antisymmetric decomposition. In this case the quasi-potential is given by:
\begin{equation}
    V(x) = -\frac{1}{2}((A+A^*)x,x),
\end{equation}
where the brackets denote an inner product.

If $A$ is not normal, then the quasipotential is given by the slightly more complicated expression:
\begin{equation}
    V(x) = -\frac{1}{2}\left( \left( \int_0^\infty \exp(At) \exp(A^*t)\,\text{d}t\right)^{-1}x, x \right).
\end{equation}
Such a quasipotential will not satisfy a normal decomposition. A useful comparison can therefore be made between this analytical expression for the quasipotential and that resulting from the SoS method for non-normal linear systems.

In [ ]:
# A 2D normal example - WORKS
n = 2;    @polyvar x[1:n]
A = [-5.0 0.2;
     0.2 -1.0];
F1(X::Vector) = A*X;
f1 = F1(x);
U1 = NormalSoS.normdecomp(f1,x, MosekSolver(),0);

(λ,u) = eig(A);
(~,v) = eig(A');
M = inv(u)*v
for i=1:n, j=1:n
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V1 = NormalSoS.filterterms(0.25*dot(inv(u*M*inv(v))*x,x));

In [27]:
@show(U1);    @show(V1);    @show(λ);

U1 = 2.499999995607779x1^2 - 0.19999999229723395x1x2 + 0.49999999447044546x2^2 + 2.69039587883021
V1 = 2.5x1^2 - 0.2x1x2 + 0.5x2^2
λ = [-5.00998, -0.990025]


In [ ]:
# A 3D normal example - WORKS
n = 3;    @polyvar x[1:n]
A = [-5.0 0.0 0.2;
     0.0 -1.5 0.0;
     0.2 0.0 -1.0];
F2(X::Vector) = A*X;
f2 = F2(x);
U2 = NormalSoS.normdecomp(f2,x, MosekSolver(),0);

(λ,u) = eig(A);
(~,v) = eig(A');
M = inv(u)*v
for i=1:n, j=1:n
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V2 = NormalSoS.filterterms(0.25*dot(inv(u*M*inv(v))*x,x));

In [30]:
@show(U2);    @show(V2);    @show(λ);

U2 = 2.4999998459624364x1^2 - 0.1999996920830655x1x3 + 0.7499999729568393x2^2 + 0.4999997328638468x3^2 + 1.799493653406083
V2 = 2.4999999999999996x1^2 - 0.1999999999999999x1x3 + 0.75x2^2 + 0.5x3^2
λ = [-5.00998, -1.5, -0.990025]


In [ ]:
# A 2D non-normal example - WORKS
n = 2;    @polyvar x[1:n]
A = [-5.0 3.8;
     0.2 -1.0];
F3(X::Vector) = A*X;
f3 = F3(x);
U3 = NormalSoS.normdecomp(f3,x, MosekSolver(),0);

(λ,u) = eig(A);
(~,v) = eig(A');
P = diagm(λ);
M = inv(u)*v
for i=1:n, j=1:n
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V3 = NormalSoS.filterterms(0.25*dot(inv(u*M*inv(v))*x,x));

In [32]:
@show(U3);    @show(V3);    @show(λ);
# NormalSoS.checknorm(f3,U3,x)

U3 = 1.7941171716143742x1^2 - 2.3529404775815133x1x2 + 1.2058820662574214x2^2 + 2.9510708010743074
V3 = 1.7941176470588243x1^2 - 2.3529411764705888x1x2 + 1.2058823529411766x2^2
λ = [-5.18174, -0.818258]


In [ ]:
# A 3D non-normal example - WORKS and agrees with quasipotential
n = 3;    @polyvar x[1:n]
A = [-5.0 3.0 0.2;
     -1.0 -1.5 3.0;
     0.2 -1.0 -1.0];
F4(X::Vector) = A*X;
f4 = F4(x);
U4 = NormalSoS.normdecomp(f4,x, MosekSolver(),0);

(λ,u) = eig(A);
(~,v) = eig(A');
M = inv(u)*v
for i=1:n, j=1:n
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V4 = NormalSoS.filterterms(0.25*dot(inv(u*M*inv(v))*x,x));

In [34]:
@show(U4);    @show(V4);    @show(λ);

U4 = 1.8342917730973012x1^2 - 1.749788550729352x1x2 + 0.18785624780097818x1x3 + 0.9937841317069048x2^2 - 0.7118287002496625x2x3 + 0.9219234571078182x3^2 + 2.010661893647271
V4 = (1.8342920720248248 + 9.656330165015786e-17im)x1^2 + (-1.7497888702168567 - 4.089691322452249e-17im)x1x2 + (0.1878561059350676 - 2.8686731123587087e-17im)x1x3 + (0.9937843646267177 + 3.467670518240395e-17im)x2^2 + (-0.7118288495863689 - 1.0674447438687807e-16im)x2x3 + (0.9219235633484566 + 3.828799556799836e-17im)x3^2
λ = Complex{Float64}[-3.94825+0.0im, -1.77587+1.66826im, -1.77587-1.66826im]


In [ ]:
# A 5D non-normal example with oscillatory dynamics - Orthogonal but does not match quasipotential
n = 5;    @polyvar x[1:n]
A = [-5.0 3.0 0.2 0.0 1.5;
     0.0 -1.5 0.0 0.0 0.5;
     -0.2 0.0 -1.0 0.0 1.0;
     0.0 1.2 0.0 -0.5 0.0;
     1.5 0.0 -3.0 0.0 -1.0];
F5(X::Vector) = A*X;
f5 = F5(x);
U5 = NormalSoS.normdecomp(f5,x, MosekSolver(),0);

(λ,u) = eig(A);
(~,v) = eig(A');
M = inv(u)*v
for i=1:n, j=1:n
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V5 = NormalSoS.filterterms(0.25*dot(inv(u*M*inv(v))*x,x));

In [ ]:
@show(U5);    @show(V5);    @show(λ);
plt5 = NormalSoS.plotlandscape(f5,U5,x,([-3 3],[-3 3]), false);    plot(plt5)

In [144]:
Ag = convert(Array,VectorOfArray(coefficients.(-differentiate(U5,x))))';
Ac = convert(Array,VectorOfArray(coefficients.(f5 + differentiate(U5,x))))';
Ag*Ac

5×5 Array{Float64,2}:
  1.59306e-5  -2.7674      -0.568846     0.421922     0.746946  
  2.76738      4.26543e-6  -0.784909     0.0242822   -1.08086   
  0.56884      0.784912     5.48259e-7  -0.0834274   -1.44943   
 -0.421922    -0.0242827    0.0834273    2.62217e-7   0.199188  
 -0.746955     1.08086      1.44944     -0.199188     1.42846e-6

In [23]:
# Display simulations of the 5 state system with damped oscillatory dynamics
using DifferentialEquations

u0 = [-2.5;2.5;1.0;1.0;10.0];
tspan = (0.0,10.0);
f(u,p,t) = F5(u);
prob = ODEProblem(f,u0,tspan);
sol = DifferentialEquations.solve(prob);

plot(plt5)
plot!(sol[1,:],sol[2,:], color=:black, legend=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 x1 
 
 
 x2 
 
 
<polyline clip-path="url(#clip4102)" style="stroke:#470b69; stroke-width:1; stroke-opacity:1; fill:none" points="
 142.324,360.065 140.193,356.624 140.149,356.548 138.282,353.03 136.676,349.678 136.601,349.512 135.158,345.995 133.865,342.477 133.158,340.332 132.733,338.959 
 131.758,335.441 130.902,331.924 130.161,328.406 129.64,325.529 129.531,324.888 129.018,321.37 128.598,317.853 128.267,314.335 128.02,310.817 127.853,307.3 
 127.763,303.782 127.746,300.264 127.799,296.746 127.919,293.229 128.104,289.711 128.349,286.193 128.654,282.676 129.015,279.158 129.43,275.64 129.64,274.067 
 129.905,272.122 130.438,268.605 131.019,265.087 131.648,261.569 132.322,258.052 133.04,254.534 133.158,253.991 133.818,251.016 134.64,247.498 135.502,243.981 
 136.402,240.463 136.676,239.444 137.357,236.945 138.356,233.427 139.389,229.91 140.193,227.261 140.462,226.392 141.59,222.874 142.748,219.357 143.711,216.512 
 143.942,215.839 145.193,212.321 146.47,208.803 147.229,206.767 147.789,205.286 149.155,201.768 150.546,198.25 150.747,197.755 151.993,194.733 153.468,191.215 
 154.264,189.353 154.984,187.697 156.543,184.179 157.782,181.424 158.13,180.662 159.771,177.144 161.3,173.905 161.433,173.626 163.155,170.108 164.817,166.745 
 164.895,166.591 166.696,163.073 168.335,159.9 168.516,159.555 170.396,156.038 171.853,153.335 172.299,152.52 174.255,149.002 175.371,147.016 176.245,145.484 
 178.276,141.967 178.888,140.919 180.355,138.449 182.406,135.021 182.461,134.931 184.632,131.414 185.924,129.336 186.834,127.896 189.077,124.378 189.441,123.813 
 191.378,120.86 192.959,118.465 193.713,117.343 196.095,113.825 196.477,113.267 198.536,110.307 199.995,108.225 201.017,106.789 203.512,103.308 203.539,103.272 
 206.133,99.7541 207.03,98.545 208.773,96.2363 210.548,93.8999 211.461,92.7186 214.066,89.37 214.199,89.2009 217.01,85.6832 217.583,84.9701 219.879,82.1654 
 221.101,80.6823 222.807,78.6477 224.619,76.5008 225.797,75.13 228.136,72.4236 228.851,71.6123 231.654,68.4488 231.974,68.0946 235.17,64.5768 235.172,64.5749 
 238.452,61.0591 238.69,60.8055 241.813,57.5414 242.207,57.1312 245.258,54.0237 245.725,53.5504 248.792,50.506 249.243,50.0615 252.422,46.9882 252.76,46.663 
 256.153,43.4705 256.278,43.3534 259.796,40.1351 260,39.9528 263.314,37.0071 263.972,36.4351 266.831,33.9656 268.074,32.9174 270.349,31.009 272.315,29.3996 
 273.867,28.1359 276.705,25.8819 277.384,25.345 280.902,22.6407 281.273,22.3642 284.42,20.0292 286.058,18.8465 287.938,17.4974 291.045,15.3287 291.455,15.044 
 294.973,12.6863 296.322,11.811 
 "/>
<polyline clip-path="url(#clip4102)" style="stroke:#63146e; stroke-width:1; stroke-opacity:1; fill:none" points="
 262.803,11.811 259.796,14.4323 258.788,15.3287 256.278,17.5703 254.877,18.8465 252.76,20.7835 251.066,22.3642 249.243,24.0729 247.348,25.8819 245.725,27.4397 
 243.72,29.3996 242.207,30.8849 240.175,32.9174 238.69,34.4098 236.709,36.4351 235.172,38.0154 233.319,39.9528 231.654,41.7031 230.001,43.4705 228.136,45.474 
 226.751,46.9882 224.619,49.3294 223.565,50.506 221.101,53.2705 220.441,54.0237 217.583,57.2987 217.375,57.5414 214.374,61.0591 214.066,61.4224 211.433,64.5768 
 210.548,65.6427 208.544,68.0946 207.03,69.9561 205.704,71.6123 203.512,74.364 202.911,75.13 200.169,78.6477 199.995,78.8727 197.486,82.1654 196.477,83.4973 
 194.845,85.6832 192.959,88.2234 192.244,89.2009 189.687,92.7186 189.441,93.06 187.187,96.2363 185.924,98.0253 184.721,99.7541 182.406,103.1 182.289,103.272 
 179.915,106.789 178.888,108.32 177.574,110.307 175.371,113.66 175.264,113.825 173.01,117.343 171.853,119.16 170.786,120.86 168.594,124.378 168.335,124.798 
 166.453,127.896 164.817,130.606 164.337,131.414 162.266,134.931 161.3,136.588 160.229,138.449 158.222,141.967 157.782,1

In [151]:
n = 10;    @polyvar x[1:n]
A = [-5.0 3.0 0.2 0.0 1.5 -5.0 3.0 0.2 0.0 1.5;
     0.0 -1.5 0.0 0.0 0.5 0.0 -1.5 0.0 0.0 0.5;
     -0.2 0.0 -1.0 0.0 1.0 -0.2 0.0 -1.0 0.0 1.0;
     0.0 1.2 0.0 -0.8 0.0 0.0 1.2 0.0 -0.5 0.0;
     1.5 0.0 -3.0 0.0 -1.0 1.5 0.0 -3.0 0.0 -1.0;
     -5.0 3.0 0.2 0.0 1.5 -5.0 3.0 0.2 0.0 1.5;
     0.0 -1.5 0.0 0.0 0.5 -0.6 -1.5 0.0 -0.2 0.8;
     -0.2 0.0 -1.0 0.0 1.0 -0.2 0.4 -1.0 0.0 1.0;
     1.5 0.0 -3.0 0.0 -1.0 1.1 -0.9 -3.0 -2.2 -1.0;
     0.0 1.2 0.0 -0.5 0.0 0.0 1.2 -2.0 -0.5 -3.4];
(λ,~) = eig(A);
λ

10-element Array{Complex{Float64},1}:
     -10.0545+0.0im     
      -4.1809+0.0im     
     -1.37426+1.90602im 
     -1.37426-1.90602im 
     -2.36019+1.03629im 
     -2.36019-1.03629im 
    0.0237035+0.124341im
    0.0237035-0.124341im
 -2.93391e-15+0.0im     
    -0.743082+0.0im     

In [127]:
M = randn(5,5);
A = M*M';
(λ,~) = eig(A)    # Eigenvalues always positive real
λ
A*A' - A'*A    # Matrix A is always normal

5×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [180]:
S = -diagm(rand(Complex{Float64},5))
A = eye(5);
for ii=1:5
    v = rand(Float64,5);
    v = v./norm(v);
    A[:,ii] = v;
end
A = A*S*A';
(λ,~) = eig(A)    # Eigenvalues all complex, unique and with positive real part
A*A' - A'*A
eig(A)[1]
A

5×5 Array{Complex{Float64},2}:
 -0.676756-0.555784im  -0.681746-0.535182im  …  -0.394976-0.114901im 
 -0.681746-0.535182im  -0.696956-0.518357im     -0.375015-0.112052im 
 -0.348316-0.335805im  -0.351562-0.330623im     -0.154341-0.0600418im
 -0.640806-0.282079im  -0.645385-0.277166im     -0.451568-0.0740618im
 -0.394976-0.114901im  -0.375015-0.112052im     -0.355153-0.0360251im

In [117]:
randn(5,5)

5×5 Array{Float64,2}:
 -0.755436   0.273702  -0.616542   -0.461627  -0.198144
  0.210334   0.412423  -0.0701546   0.351034   0.117369
 -0.120889  -0.825627   0.178306    0.200714   0.276668
 -0.748894  -0.444011  -0.629517    0.145496   0.847733
 -0.170601   1.20426    1.64747    -1.15534    0.460915